Run in Colab development environment.



In [2]:
import torch
import torchvision.io as io

In [1]:
#Use for Drive files

import os
from google.colab import drive
drive.mount('/content/drive')

Current working directory: /content
Mounted at /content/drive


https://pytorch.org/vision/main/models.html

In [3]:
from torchvision.io import decode_image

from torchvision.models import swin_t, Swin_T_Weights
from torchvision.models import swin_s, Swin_S_Weights
from torchvision.models import swin_b, Swin_B_Weights
from torchvision.models import swin_v2_t, Swin_V2_T_Weights
from torchvision.models import swin_v2_s, Swin_V2_S_Weights
from torchvision.models import swin_v2_b, Swin_V2_B_Weights

Images with Synthetic Object Added

In [8]:
def decode_multiple_images(image_paths): # Decodes multiple images from given paths using torchvision.
    image_tensors = []
    for path in image_paths:
        try:
            image_bytes = io.read_file(path)
            image_tensor = io.decode_image(image_bytes)
            if image_tensor.shape[0] == 4:
              image_tensor = image_tensor[:3, :, :]
            image_tensors.append(image_tensor)
        except Exception as e:
            print(f"Error decoding image {path}: {e}")
    return torch.stack(image_tensors) if image_tensors else None

image_folder = "/content/drive/MyDrive/" #image path

image_files = [os.path.join(image_folder, f) for f in os.listdir(image_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

decoded_images = decode_multiple_images(image_files)

if decoded_images is not None:
    print("Decoded images tensor shape:", decoded_images.shape) #Returns: torch.Tensor with the decoded images.
else:
    print("No images were successfully decoded.")

Decoded images tensor shape: torch.Size([100, 3, 512, 1392])


In [7]:
# Swin Model series

#weights = Swin_T_Weights.DEFAULT
#model = swin_t(weights=weights)
#weights = Swin_S_Weights.DEFAULT
#model = swin_s(weights=weights)
#weights = Swin_B_Weights.DEFAULT
#model = swin_b(weights=weights)
#weights = Swin_V2_T_Weights.DEFAULT
#model = swin_v2_t(weights=weights)
#weights = Swin_V2_S_Weights.DEFAULT
#model = swin_v2_s(weights=weights)#
weights = Swin_V2_B_Weights.DEFAULT
model = swin_v2_b(weights=weights)

model.eval()

Downloading: "https://download.pytorch.org/models/swin_v2_b-781e5279.pth" to /root/.cache/torch/hub/checkpoints/swin_v2_b-781e5279.pth
100%|██████████| 336M/336M [00:04<00:00, 82.8MB/s]


SwinTransformer(
  (features): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
      (1): Permute()
      (2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
    )
    (1): Sequential(
      (0): SwinTransformerBlockV2(
        (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (attn): ShiftedWindowAttentionV2(
          (qkv): Linear(in_features=128, out_features=384, bias=True)
          (proj): Linear(in_features=128, out_features=128, bias=True)
          (cpb_mlp): Sequential(
            (0): Linear(in_features=2, out_features=512, bias=True)
            (1): ReLU(inplace=True)
            (2): Linear(in_features=512, out_features=4, bias=False)
          )
        )
        (stochastic_depth): StochasticDepth(p=0.0, mode=row)
        (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (0): Linear(in_features=128, out_features=512, bias=True)
          (1): GELU(appro

https://pytorch.org/vision/main/models.html

In [10]:
img = decoded_images

# Initialize
preprocess = weights.transforms()

# Apply inference preprocessing transforms **and prediction for each image
for i in range(img.shape[0]):  # Iterate through each image in decoded_images
    single_img = img[i]  # Get a single image from the tensor

    # Convert the image to RGB if it has 4 channels (RGBA)
    if single_img.shape[0] == 4:
        single_img = single_img[:3, :, :]  # Select only the first 3 channels (RGB)

    batch = preprocess(single_img).unsqueeze(0)

    # Use the model to predict category
    prediction = model(batch).squeeze(0).softmax(0)
    class_id = prediction.argmax().item()
    score = prediction[class_id].item() #score/confidence of prediction

    category_name = weights.meta["categories"][class_id]
    print(f"{category_name}: {100 * score:.1f}%")

    with open(os.path.join(image_folder, 'testrun.csv'), 'a') as file:
      file.write(category_name + ',' + str(100*score) + '\n') # Concatenate the strings into a single string before writing

bicycle-built-for-two: 32.4%
bicycle-built-for-two: 38.6%
bicycle-built-for-two: 52.5%
bicycle-built-for-two: 75.8%
bicycle-built-for-two: 69.5%
bicycle-built-for-two: 41.0%
bicycle-built-for-two: 50.8%
bicycle-built-for-two: 39.2%
bicycle-built-for-two: 45.7%
bicycle-built-for-two: 48.4%
bicycle-built-for-two: 33.4%
bicycle-built-for-two: 80.5%
bicycle-built-for-two: 53.8%
bicycle-built-for-two: 31.8%
bicycle-built-for-two: 71.6%
bicycle-built-for-two: 58.7%
bicycle-built-for-two: 20.0%
tricycle: 43.0%
bicycle-built-for-two: 47.9%
tricycle: 49.2%
bicycle-built-for-two: 82.7%
bicycle-built-for-two: 39.7%
bicycle-built-for-two: 47.8%
bicycle-built-for-two: 56.3%
bicycle-built-for-two: 73.4%
bicycle-built-for-two: 24.5%
bicycle-built-for-two: 57.6%
tricycle: 74.9%
tricycle: 28.3%
bicycle-built-for-two: 38.7%
bicycle-built-for-two: 45.9%
bicycle-built-for-two: 31.2%
bicycle-built-for-two: 54.4%
bicycle-built-for-two: 64.4%
bicycle-built-for-two: 58.8%
tricycle: 46.4%
bicycle-built-for-two